In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import string
import re
import gensim
from gensim.models import Word2Vec
from gensim.models import FastText
import gensim.downloader as api
print((gensim.__version__))  # needed 3.8.3   -> pip install gensim==3.8.3
import pickle
import time

4.1.2


In [2]:
def tokenized_column(string_col
                    ):  
    tokenized_list=[]
    for i in list(string_col):
        li = list(i.split(' '))
        tokenized_list.append(li)
    return tokenized_list

# EMBEDDINGS

In [3]:
df_all = pd.read_csv("outputs/df_sw_tok_low_punc_lemm_v7.csv")

In [4]:
len(df_all)

476175

In [5]:
df_all.isnull().sum()

Unnamed: 0              0
Year                  711
Month                 711
abstract                0
doi                     0
cord_uid                0
journal             40225
license                 0
authors              1986
len                     0
language                0
abstract_cleaned        0
dtype: int64

In [6]:
df_all[df_all["Year"]==2021]

,Unnamed: 0,Year,Month,abstract,doi,cord_uid,journal,license,authors,len,language,abstract_cleaned
11062,14573,2021.0,1.0,"In April 2017, surveillance detected a surge i...",10.3201/eid2701.201347,aoiambb8,Emerg Infect Dis,no-cc,"Rahaman, Md R.; Alroy, Karen A.; Van Beneden, ...",298,en,april 2017 surveillance detected surge severe ...
11066,14577,2021.0,1.0,Carbon dioxide measurement is useful for confi...,10.1007/s10877-020-00638-0,embv1el9,J Clin Monit Comput,no-cc,"Hirakawa, Eiji; Ibara, Satoshi",960,en,carbon dioxide measurement useful confirmation...
11067,14578,2021.0,1.0,BACKGROUND: Diabetes mellitus has been commonl...,10.1007/s10151-020-02373-9,8dcgzyst,Tech Coloproctol,no-cc,"Tan, D. J. H.; Yaow, C. Y. L.; Mok, H. T.; Ng,...",1387,en,diabetes mellitus commonly associated poor sur...
11068,14579,2021.0,1.0,OBJECTIVES: A potent HIV vaccine should overco...,10.1007/s10529-020-03060-3,sgtvecj8,Biotechnol Lett,no-cc,"Davoodi, Saba; Bolhassani, Azam; Namazi, Fatemeh",1104,en,potent hiv vaccine overcome limitation polymor...
11069,14580,2021.0,1.0,This research was conducted in order to examin...,10.1007/s10943-020-01142-0,3vtq6d8f,J Relig Health,no-cc,"Aslan, Hakime; Erci, Behice; Pekince, Hatice",705,en,research conducted order examine correlation w...
...,...,...,...,...,...,...,...,...,...,...,...,...
476156,992900,2021.0,9.0,BACKGROUND: Cancer patients are at higher risk...,10.1016/j.esmoop.2021.100274,4xgndb8y,ESMO Open,no-cc,"Peeters, M.; Verbruggen, L.; Teuwen, L.; Vanho...",1639,en,cancer patient higher risk developing severe c...
476165,992910,2021.0,1.0,A workflow for rapid SARS-CoV-2 epitope discov...,10.3390/vaccines9010035,ctyevypl,Vaccines (Basel),cc-by,"Musicò, Angelo; Frigerio, Roberto; Mussida, Al...",992,en,workflow rapid sars-cov-2 epitope discovery pe...
476168,992913,2021.0,6.0,Coronavirus disease-associated mucormycosis (C...,10.4103/ijo.ijo_1439_21,lgljfzmh,Indian J Ophthalmol,cc-by-nc-sa,"Sreshta, Kanduri; Dave, Tarjani Vivek; Varma, ...",743,en,coronavirus disease-associated mucormycosis ca...
476169,992914,2021.0,2.0,We aimed to investigate whether statin therapy...,10.3390/jpm11020116,xaar9cav,J Pers Med,cc-by,"Oh, Tak Kyu; Song, In-Ae; Jeon, Young-Tae",952,en,aimed investigate whether statin therapy assoc...


## A) Train W2V with articles published between years

In [6]:
to_years = [2017,2018,2019,2020,2021]
diff_list = [1]

for diff in diff_list:
    for to_year in to_years:
        print("Training started................")
        print("Max publ. year: "+str(to_year))
        print("Min publ. year: "+str(to_year-diff))
        df_all_to_year = df_all[ (df_all['Year']<=to_year) & (df_all['Year']>=to_year-diff) ]
        print("Count of articles: " + str(len(df_all_to_year)))
        model = gensim.models.word2vec.Word2Vec(tokenized_column(df_all_to_year["abstract_cleaned"].astype(str)),
                                        vector_size=100, 
                                        window = 10,
                                        min_count=2,
                                        sg=1,
                                        seed=1
                                               ) 
        model.save("outputs/w2v/w2v_published_between_"+str(to_year-diff) + " and "+ str(to_year)+".model")

Training started................
Max publ. year: 2017
Min publ. year: 2016
Count of articles: 12589
Training started................
Max publ. year: 2018
Min publ. year: 2017
Count of articles: 13036
Training started................
Max publ. year: 2019
Min publ. year: 2018
Count of articles: 14503
Training started................
Max publ. year: 2020
Min publ. year: 2019
Count of articles: 132422
Training started................
Max publ. year: 2021
Min publ. year: 2020
Count of articles: 350176


## B) For qualitative validation  - Experiment 4+5

In [7]:
to_years = [2022]
diff_list = [3]

for diff in diff_list:
    for to_year in to_years:
        print("Training started................")
        print("Max publ. year: "+str(to_year))
        print("Min publ. year: "+str(to_year-diff))
        df_all_to_year = df_all[ (df_all['Year']<=to_year) & (df_all['Year']>=to_year-diff) ]
        print("Count of articles: " + str(len(df_all_to_year)))
        model = gensim.models.word2vec.Word2Vec(tokenized_column(df_all_to_year["abstract_cleaned"].astype(str)),
                                        vector_size=100, 
                                        window = 10,
                                        min_count=2,
                                        sg=1,
                                        seed=1
                                               ) 
        model.save("outputs/w2v/w2v_published_between_"+str(to_year-diff) + " and "+ str(to_year)+".model")

Training started................
Max publ. year: 2022
Min publ. year: 2019
Count of articles: 414174
